In [3]:
import pandas as pd

In [4]:
filename = "data/r2_survey_answers_csv.csv"

In [11]:
df_use = pd.read_csv(
    filename, 
    encoding="cp932",
    usecols=[
        "地方公共団体名",
        "提出状況",
        "[No.1]良く知っている",
        "[No.1]知っているが詳細は分からない",
        "[No.1]名前は知っている",
        "[No.1]知らない",
        "[No.2]2020年度中",
        "[No.2]2021年上期",
        "[No.2]2021年下期",
        "[No.2]未計画",
        "[No.2]オープンデータをすでに公開している",
        "[No.3]自由記述",
    ]
).set_axis(
    [
        "自治体名", 
        "提出状況",
        "良く知っている", 
        "詳細は知らない", 
        "名前は知っている", 
        "知らない",
        "2020年度中", 
        "2021年上期", 
        "2021年下期", 
        "未計画", 
        "公開済み",
        "自由記述"
    ],
    axis=1
).iloc[47:, :]

In [12]:
df_use.shape

(1741, 12)

In [13]:
df_use.head()

,自治体名,提出状況,良く知っている,詳細は知らない,名前は知っている,知らない,2020年度中,2021年上期,2021年下期,未計画,公開済み,自由記述
47,北海道 札幌市,提出済,1,0,0,0,0,0,0,0,1,NaN
48,北海道 函館市,未提出,0,0,0,0,0,0,0,0,0,NaN
49,北海道 小樽市,提出済,1,0,0,0,0,0,0,0,1,NaN
50,北海道 旭川市,提出済,1,0,0,0,0,0,0,0,1,NaN
51,北海道 室蘭市,提出済,1,0,0,0,0,0,0,0,1,NaN


In [14]:
def get_category(row):
    if row.sum() == 0:
        return '未回答'
    else:
        return row.idxmax()

In [21]:
popularity = pd.CategoricalDtype(
    categories=[
        "良く知っている", 
        "詳細は知らない", 
        "名前は知っている", 
        "知らない",
    ],
    ordered=True,
)

In [22]:
df = pd.concat(
        [
         df_use.loc[:, "自治体名"].str.split(" ", expand=True).astype("string"),
         df_use.loc[:, "提出状況"].astype("category"),
         df_use.loc[:, "良く知っている":"知らない"].apply(get_category, axis=1).astype(popularity),
         df_use.loc[:, "2020年度中":"公開済み"].apply(get_category, axis=1).astype("category"),
         df_use.loc[:, "自由記述"],
        ],
    axis=1,
).set_axis(["都道府県", "市町村", "提出状況", "知名度", "公開状況", "自由記述"], axis=1)

In [23]:
df.head()

,都道府県,市町村,提出状況,知名度,公開状況,自由記述
47,北海道,札幌市,提出済,良く知っている,公開済み,NaN
48,北海道,函館市,未提出,NaN,未回答,NaN
49,北海道,小樽市,提出済,良く知っている,公開済み,NaN
50,北海道,旭川市,提出済,良く知っている,公開済み,NaN
51,北海道,室蘭市,提出済,良く知っている,公開済み,NaN


In [24]:
df.dtypes

都道府県    string[python]
市町村     string[python]
提出状況          category
知名度           category
公開状況          category
自由記述            object
dtype: object

In [25]:
df.to_pickle("df-sumarry.pkl")